In [1]:
import pennylane as qml
import numpy as np
from pennylane import numpy as pnp
try:
    from jax import numpy as jnp
    import jax
    jax.config.update("jax_enable_x64", True)
except ImportError:
    print("current environment does not have jax installed")

current environment does not have jax installed


In [2]:
from qiskit_ibm_provider import IBMProvider

In [3]:
from qiskit_ibm_runtime import QiskitRuntimeService, Session, Options

service = QiskitRuntimeService(channel="ibm_quantum")

## Testing IBMQ

In [7]:
#from pennylane_qiskit.qiskit_device2 import QiskitDevice2
from pennylane_qiskit.qiskit_device import QiskitDevice

use_simulator = False

if use_simulator:
    backend = service.backend("ibmq_qasm_simulator")
    dev = qml.device('qiskit.remote', wires=2, backend=backend, use_primitives=True)

    print("RUNNING ON SIMULATOR")
else:
    backend = service.least_busy(simulator=False, operational=True)
    dev = qml.device('qiskit.remote', wires=2, backend=backend, use_primitives=True)
    
    print(f"Running on {backend.name}")
    
session = Session(backend=backend)

Running on ibm_sherbrooke


In [8]:
@qml.qnode(dev)
def circuit(x):
    qml.RX(x, wires=[0])
    qml.CNOT(wires=[0, 1])
    return qml.expval(qml.PauliZ(0))

circuit(np.pi/2)

/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/contextlib.py:81: UserWarning: use_primitives is not a recognized runtime option and may be ignored by the backend.
  return func(*args, **kwds)


In [11]:
res = circuit(np.pi/2, shots=50)
print(res)

/Users/austin.huang/pennylane/pennylane-qiskit/venv/lib/python3.12/site-packages/qiskit_ibm_runtime/qiskit_runtime_service.py:915: UserWarning: Cloud simulators have been deprecated and will be removed on 15 May 2024. Use the new local testing mode in qiskit-ibm-runtime version 0.22.0 or later to meet your debugging needs.
  warnings.warn(warning_message)


[(tensor([[0, 0, 0],
        [0, 0, 0]], requires_grad=True), tensor([[0, 0, 0],
        [0, 0, 0]], requires_grad=True))]


In [5]:
qiskit_dev = QiskitDevice2(wires=3, backend=backend, shots=10, use_primitives=True)

@qml.qnode(dev)
def circuit():
    return [qml.expval(qml.PauliZ(0))]

res = circuit(shots=[5])


/Users/austin.huang/pennylane/pennylane-qiskit/venv/lib/python3.12/site-packages/qiskit_ibm_runtime/qiskit_runtime_service.py:915: UserWarning: Cloud simulators have been deprecated and will be removed on 15 May 2024. Use the new local testing mode in qiskit-ibm-runtime version 0.22.0 or later to meet your debugging needs.
  warnings.warn(warning_message)


In [6]:
print(res)


[(tensor(1., requires_grad=True),)]


In [ ]:
hamiltonian = qml.Hamiltonian([1, 3], [qml.PauliX(0), qml.PauliZ(0) @ qml.PauliZ(1)])
@qml.qnode(dev)
def circuit():
    return qml.expval(hamiltonian)

print(hamiltonian)
circuit()

In [ ]:
op = qml.PauliX(0)
op2 = qml.prod(qml.PauliZ(0), qml.PauliZ(1))
op2 = qml.s_prod(1.24, op2)
op = qml.sum(op, op2)

@qml.qnode(dev)
def circuit():
    return qml.expval(op)

print(op)
op.terms()[1][1].terms()

In [ ]:
@qml.qnode(dev)
def circuit(x):
    qml.sum(qml.PauliY(1), qml.PauliZ(0)),
    return qml.expval(qml.sum(qml.PauliZ(0), qml.PauliX(1)))

circuit(np.pi/2)

In [ ]:
@qml.qnode(dev)
def circuit_batched(x, y):
    qml.RX([x, y], wires=[0])
    qml.CNOT(wires=[0, 1])
    return qml.expval(qml.PauliZ(1)), qml.probs(), qml.var(qml.PauliZ(0)), qml.expval(qml.PauliY(0)), qml.sample()

@qml.qnode(dev)
def circuit(x):
    qml.RX(x, wires=[0])
    qml.CNOT(wires=[0, 1])
    return qml.expval(qml.PauliZ(1))

@qml.qnode(dev)
def circuit_counts(x):
    qml.RX(x, wires=[0])
    qml.CNOT(wires=[0, 1])
    return qml.counts()

In [ ]:
circuit(1)

In [ ]:
circuit_batched(0.1, np.pi)

In [ ]:
from pennylane_qiskit.qiskit_device2 import qiskit_session

angle = 0.1
i = 0

with qiskit_session(dev) as session:
    
    res = circuit(angle)[0]
    print(res)

    while res > 0:
        angle += 0.5
        i += 1
        res = circuit(angle)[0]
        print(res)
        
    print(circuit_counts(angle))
        
i, angle, res

## Testing Runtime devices directly in Qiskit for debugging

In [ ]:
from qiskit import QuantumCircuit

# define circuit
def circuit(x):
    qc = QuantumCircuit(2, 2)
    qc.rx(x, 0)
    qc.cx(0, 1)
    qc.measure(1, 0)
    return qc

qc = circuit(1.2)
qc.draw('text')

In [ ]:
from qiskit_ibm_runtime import Sampler

res = []

with Session(backend=backend, max_time="1m"):
    sampler = Sampler()

    for qc in [circuit(0.1), circuit(np.pi/2)]:
        result = sampler.run(qc).result()
        res.append(result.quasi_dists[0])

In [ ]:
from qiskit.primitives import Estimator

estimator = Estimator()
estimator.run?